In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

# Fetch stock data (e.g., Apple)
stock_data = yf.download('AAPL', start='2020-01-01', end='2023-01-01')
# Calculate daily price change (percentage)
stock_data['Price_Change'] = stock_data['Close'].pct_change() * 100
stock_data = stock_data.dropna()
price_data = stock_data[['Close', 'Price_Change']].values

/tmp/ipykernel_80651/1157414683.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download('AAPL', start='2020-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed


In [5]:
# Example: Dummy textual data (replace with real data)
text_data = pd.DataFrame({
    'Date': stock_data.index,
    'Headline': ['Apple launches new iPhone', 'Market crash fears']  # Replace with real headlines
})

ValueError: All arrays must be of the same length

In [7]:
stock_data = stock_data.reset_index()
combined_data = pd.merge(stock_data[['Date', 'Close', 'Price_Change']], 
                       text_data[['Date', 'Headline']], 
                       on='Date', 
                       how='inner')

NameError: name 'text_data' is not defined

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_price_data = scaler.fit_transform(combined_data[['Close', 'Price_Change']])

NameError: name 'combined_data' is not defined

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_price_data = scaler.fit_transform(combined_data[['Close', 'Price_Change']])

NameError: name 'combined_data' is not defined

In [13]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])  # Sequence of 60 days
        y.append(data[i + seq_length, 0])  # Predict next day's closing price
    return np.array(X), np.array(y)

seq_length = 60
X_price, y_price = create_sequences(scaled_price_data, seq_length)

NameError: name 'scaled_price_data' is not defined

In [15]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
combined_data['Sentiment'] = combined_data['Headline'].apply(
    lambda x: sid.polarity_scores(x)['compound']  # Compound score: -1 to 1
)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/labsi/nltk_data...


NameError: name 'combined_data' is not defined

In [17]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # CLS token embedding

combined_data['Text_Embedding'] = combined_data['Headline'].apply(get_bert_embedding)

ModuleNotFoundError: No module named 'transformers'

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, X_combined.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=25))
model.add(Dense(units=1))  # Predict next day's closing price
model.compile(optimizer='adam', loss='mean_squared_error')

2025-09-08 16:02:13.093983: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-08 16:02:13.094217: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-08 16:02:13.127043: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-08 16:02:13.884908: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation or

NameError: name 'X_combined' is not defined

In [21]:
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.models import Model

# Numerical input branch
price_input = Input(shape=(seq_length, 2))  # Price and Price_Change
x = LSTM(50, return_sequences=True)(price_input)
x = Dropout(0.2)(x)
x = LSTM(50)(x)
x = Dropout(0.2)(x)

# Text input branch (e.g., BERT embeddings)
text_input = Input(shape=(768,))  # BERT embedding size
y = Dense(50, activation='relu')(text_input)
y = Dropout(0.2)(y)

# Concatenate
combined = Concatenate()([x, y])
z = Dense(25, activation='relu')(combined)
output = Dense(1)(z)

model = Model(inputs=[price_input, text_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

2025-09-08 16:02:20.655178: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
model.fit(X_combined, y_combined, epochs=50, batch_size=32, validation_split=0.2)